In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4336/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4336/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_35_2,0.534803,0.557097,0.716108,0.481931,0.126524,0.262926,0.241802,0.224888,0.179078,0.355702,0.366540,0.361304,38.134648,75.099232,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_35_6,0.534841,0.556918,0.714588,0.479659,0.126513,0.263033,0.243097,0.225874,0.179117,0.355687,0.366592,0.361289,38.134814,75.099398,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_35_9,0.534779,0.556568,0.711726,0.475295,0.126530,0.263240,0.245535,0.227768,0.179173,0.355711,0.366508,0.361313,38.134547,75.099131,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_1_36_2,0.536109,0.554113,0.708839,0.558475,0.126169,0.264698,0.247994,0.071591,0.179381,0.355202,0.368318,0.360796,38.140271,75.104855,"Hidden Size=[4], regularizer=0.02, learning_ra..."
4,model_1_34_7,0.537151,0.550700,0.710412,-0.078106,0.125885,0.266724,0.246654,0.148283,0.179286,0.354803,0.369738,0.360391,38.144771,75.109354,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,model_3_26_8,0.325126,-211.118550,-180.943920,-1.445726,0.183551,125.922735,154.969104,0.652666,0.169717,0.428429,-1.879461,0.435176,101.390521,207.935497,"Hidden Size=[12], regularizer=0.02, learning_r..."
560,model_3_26_2,0.463744,-221.223876,-226.248405,0.005729,0.145850,131.921692,193.556793,0.265331,0.178973,0.381904,-1.288025,0.387918,101.850349,208.395325,"Hidden Size=[12], regularizer=0.02, learning_r..."
561,model_3_27_2,-0.365842,-222.935340,-259.778514,-65.687114,0.371480,132.937691,222.115763,1.461432,0.194809,0.609492,-4.827591,0.619091,99.980520,206.525496,"Hidden Size=[12], regularizer=0.02, learning_r..."
562,model_3_25_9,0.409350,-224.694138,-233.073374,-0.649494,0.160644,133.981790,199.369899,0.443521,0.177187,0.400805,-1.520108,0.407117,101.657123,208.202100,"Hidden Size=[12], regularizer=0.02, learning_r..."
